# Confidence plot

**Goal**:
1) Use the BaggingRegressor to compute the standard deviation of predictions
2) Create the confidence plot

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition import ElementProperty
plt.rcParams['figure.dpi'] = 120
plt.rcParams['axes.grid'] = False
print('Imports OK')


## 1) Load dataset

In [ ]:
csv_path = Path('..') / 'data' / 'DS1.csv'
assert csv_path.exists(), f"Could not find {csv_path}. Adjust the path if you moved the file."

# Load data
df = pd.read_csv(csv_path)

## 3) Train/Test split (use train for EDA and CV comparison)
#### We also split the names for later use

In [ ]:
y_all = df['TC']
names = df['Name'] 
X_orig_all = df.drop(columns=['TC']).select_dtypes(include=[np.number])
mask = y_all.notna()
y_all = y_all.loc[mask]
X_orig_all = X_orig_all.loc[mask]
X_orig_train, X_orig_test, y_train, y_test, names_train, names_test = train_test_split(
    X_orig_all, y_all, names, test_size=0.30, random_state=123, shuffle=True
)
X_orig_train.shape

## 5) Fit with bagging regressor

In [ ]:
reg = BaggingRegressor(RandomForestRegressor(random_state = 123),
                             n_estimators=20, random_state=123)
reg.fit(X_orig_train, y_train)
y_pred_orig = reg.predict(X_orig_test)
X_test_np  = X_orig_test.to_numpy()  if hasattr(X_orig_test, "to_numpy")  else X_orig_test
predictions = np.array([estimator.predict(X_test_np) for estimator in reg.estimators_])
std = np.std(predictions, axis=0)
def report(y_true, y_pred, label):
    r2  = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f'{label} - R^2: {r2:.4f}, MAE: {mae:.3f}')
report(y_test, y_pred_orig, 'TEST RF (original)')

## 6) Confidence plot
We now have the following  
*y_pred_orig* — ensemble prediction (mean over estimators)  
*std* — the per‑sample uncertainty proxy (std over estimator predictions)  
*y_test* — ground truth

### Code and helper functions for the confidence plot

In [ ]:
def _as_np(x):
    # Helper to make sure we have numpy arrays
    return x.to_numpy().ravel() if hasattr(x, "to_numpy") else np.asarray(x).ravel()

def confidence_curve(
    y_true,
    y_pred,
    uncertainty,
    metric: str = "mae",
    step: int = 1,
    min_remaining: int = 10,
):
    """
    Compute a confidence curve.

    Parameters
    ----------
    y_true : array-like, shape (n_samples,)
        Ground-truth targets.
    y_pred : array-like, shape (n_samples,)
        Model predictions.
    uncertainty : array-like, shape (n_samples,)
        Per-sample uncertainty estimates (higher = less confident).
    metric : {"mae", "rmse"}
        Error metric used to evaluate the remaining subset.
    step : int
        Number of samples to discard per iteration (use >1 for smoother/faster curves).
    min_remaining : int
        Stop discarding when fewer than this number of samples would remain.

    Returns
    -------
    pct_discarded : np.ndarray
        Percent of discarded samples at each step (0..100).
    err_curve : np.ndarray
        Error metric value on the remaining subset at each step.
    """
    y_true = _as_np(y_true)
    y_pred = _as_np(y_pred)
    uncertainty = _as_np(uncertainty)

    assert y_true.shape == y_pred.shape == uncertainty.shape

    n = len(y_true)
    order = np.argsort(-uncertainty)  # descending uncertainty
    y_true_sorted = y_true[order]
    y_pred_sorted = y_pred[order]

    def compute_error(y_t, y_p):
        if metric.lower() == "mae":
            return mean_absolute_error(y_t, y_p)
        elif metric.lower() == "rmse":
            return np.sqrt(mean_squared_error(y_t, y_p))
        else:
            raise ValueError("metric must be 'mae' or 'rmse'")

    discards = list(range(0, max(0, n - min_remaining + 1), step))
    pct_discarded = []
    err_curve = []

    for k in discards:
        # Discard k most-uncertain samples: evaluate on the remaining tail
        y_t_rem = y_true_sorted[k:]
        y_p_rem = y_pred_sorted[k:]
        if len(y_t_rem) < min_remaining:
            break
        pct_discarded.append(100.0 * k / n)
        err_curve.append(compute_error(y_t_rem, y_p_rem))

    return np.array(pct_discarded), np.array(err_curve)


def ideal_curve(y_true, y_pred, metric: str = "mae", step: int = 1, min_remaining: int = 10):
    """
    Oracle curve: same as confidence_curve but discard by true absolute error.
    """
    y_true = _as_np(y_true)
    y_pred = _as_np(y_pred)

    abs_err = np.abs(y_true - y_pred)
    # Highest actual error first
    order = np.argsort(-abs_err)

    n = len(y_true)
    y_true_sorted = y_true[order]
    y_pred_sorted = y_pred[order]

    def compute_error(y_t, y_p):
        if metric.lower() == "mae":
            return mean_absolute_error(y_t, y_p)
        elif metric.lower() == "rmse":
            return np.sqrt(mean_squared_error(y_t, y_p))
        else:
            raise ValueError("metric must be 'mae' or 'rmse'")

    discards = list(range(0, max(0, n - min_remaining + 1), step))
    pct_discarded = []
    err_curve = []

    for k in discards:
        y_t_rem = y_true_sorted[k:]
        y_p_rem = y_pred_sorted[k:]
        if len(y_t_rem) < min_remaining:
            break
        pct_discarded.append(100.0 * k / n)
        err_curve.append(compute_error(y_t_rem, y_p_rem))

    return np.array(pct_discarded), np.array(err_curve)


def plot_confidence_curve(
    y_true,
    y_pred,
    uncertainty,
    metric: str = "mae",
    step: int = 1,
    min_remaining: int = 10,
    title: str = "Confidence Curve",
    ax=None,
):
    """
    Build and plot the confidence curve alongside the ideal (oracle) curve.
    """
    pct_uq, err_uq = confidence_curve(
        y_true, y_pred, uncertainty, metric=metric, step=step, min_remaining=min_remaining
    )
    pct_id, err_id = ideal_curve(
        y_true, y_pred, metric=metric, step=step, min_remaining=min_remaining
    )

    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 5))

    ax.plot(pct_uq, err_uq, label="By predicted uncertainty", color="#1f77b4", lw=2)
    ax.plot(pct_id, err_id, label="Ideal (by true error)", color="#d62728", lw=2, ls="--")

    ax.set_xlabel("Discarded samples (%)")
    ylabel = "MAE on remaining set" if metric.lower() == "mae" \
             else "RMSE on remaining set"
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    ax.legend(loc="best")
    return ax

### Now we plot the curve

In [ ]:
# Ensure arrays (and consistent indexing with y_test)
y_t = _as_np(y_test)
y_p = _as_np(y_pred_orig)
u   = _as_np(std)

# Plot with MAE (default). Use step>1 for smoother curve on big test sets.
ax = plot_confidence_curve(
    y_true=y_t,
    y_pred=y_p,
    uncertainty=u,
    metric="mae",     # or "rmse"
    step=5,           # discard in chunks of 5 samples for speed/smoothness
    min_remaining=20, # don't evaluate on very tiny remainders
    title="Bagging RF Confidence Curve (uncertainty = per-estimator std)"
)
plt.show()